In [ ]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Step 1: Load the data and initial inspection
# ----------------------------------------------
# Load data (corrected file path)
data = pd.read_csv(r'C:\Users\bhupa\OneDrive - ViaUC\Semester6\MAL1\final\MAL1_Project\archive\all_stocks_5yr.csv')

# Convert date to datetime
data['date'] = pd.to_datetime(data['date'])

# Check for missing values
missing_values = data.isnull().sum()
print("Missing Values:\n", missing_values)

# Step 2: Identify target variable
# ---------------------------------
# For this analysis, we assume 'close' is the target variable for supervised learning tasks
target = 'close'

# Step 3: Visualize the data
# ---------------------------
# Plotting time series of prices for a specific stock (e.g., 'AAL')
stock_data = data[data['Name'] == 'AAL']

plt.figure(figsize=(14, 7))
plt.plot(stock_data['date'], stock_data['close'], label='Close')
plt.plot(stock_data['date'], stock_data['open'], label='Open')
plt.plot(stock_data['date'], stock_data['high'], label='High')
plt.plot(stock_data['date'], stock_data['low'], label='Low')
plt.legend()
plt.title('Stock Prices Over Time for AAL')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

# Plotting volume over time for the same stock
plt.figure(figsize=(14, 7))
plt.plot(stock_data['date'], stock_data['volume'], label='Volume')
plt.legend()
plt.title('Trading Volume Over Time for AAL')
plt.xlabel('Date')
plt.ylabel('Volume')
plt.show()

# Step 4: Study correlations between features
# --------------------------------------------
# Correlation matrix for the specific stock
corr_matrix = stock_data[['open', 'high', 'low', 'close', 'volume']].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix for AAL')
plt.show()

# Step 5: Identify promising transformations
# -------------------------------------------
# Log transformation of volume if skewed
if stock_data['volume'].skew() > 1:
    stock_data['log_volume'] = np.log(stock_data['volume'])

# Example of creating lag feature
stock_data['prev_close'] = stock_data['close'].shift(1)

# Example of moving average
stock_data['7_day_ma'] = stock_data['close'].rolling(window=7).mean()

# Display first few rows to verify transformations
print(stock_data.head())

# Save the transformed dataset to a new CSV file for further use
stock_data.to_csv('transformed_AAL_stock_data.csv', index=False)


In [ ]:
import logging
from pathlib import Path
from typing import Tuple, Union, Optional, Dict, Any

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping
from prophet import Prophet
import pmdarima as pm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def load_data(filepath: Union[str, Path]) -> Optional[pd.DataFrame]:
    """Load stock data from a CSV file."""
    try:
        data = pd.read_csv(filepath)
        data['date'] = pd.to_datetime(data['date'])
        required_columns = {'date', 'open', 'high', 'low', 'close', 'volume', 'Name'}
        missing_columns = required_columns - set(data.columns)
        if missing_columns:
            raise ValueError(f"Missing columns: {missing_columns}")
        logger.info("Data loaded successfully.")
        return data
    except Exception as e:
        logger.error(f"Error loading data: {e}")
        return None

def explore_data(data: pd.DataFrame) -> pd.DataFrame:
    """Explore the data and log important characteristics."""
    exploration_report = []
    
    for col in data.columns:
        col_type = data[col].dtype
        missing_percentage = data[col].isnull().mean() * 100
        unique_values = data[col].nunique()
        outliers = ((np.abs(zscore(data[col])) > 3).sum()) if np.issubdtype(col_type, np.number) else 'N/A'
        exploration_report.append({
            "Name": col,
            "Type": col_type,
            "Missing Percentage": missing_percentage,
            "Unique Values": unique_values,
            "Outliers": outliers
        })
        
        logger.info(f"Feature '{col}' - Type: {col_type}, Missing: {missing_percentage:.2f}%, Unique: {unique_values}, Outliers: {outliers}")
    
    exploration_df = pd.DataFrame(exploration_report)
    logger.info(f"Exploration Report:\n{exploration_df}")
    return exploration_df

def visualize_data(stock_data: pd.DataFrame, stock_name: str) -> None:
    """Visualize stock prices and trading volume over time."""
    plt.figure(figsize=(14, 7))
    plt.plot(stock_data['date'], stock_data['close'], label='Close')
    plt.plot(stock_data['date'], stock_data['open'], label='Open')
    plt.plot(stock_data['date'], stock_data['high'], label='High')
    plt.plot(stock_data['date'], stock_data['low'], label='Low')
    plt.legend()
    plt.title(f'Stock Prices Over Time: {stock_name}')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.show()

    plt.figure(figsize=(14, 7))
    plt.plot(stock_data['date'], stock_data['volume'], label='Volume')
    plt.legend()
    plt.title(f'Trading Volume Over Time: {stock_name}')
    plt.xlabel('Date')
    plt.ylabel('Volume')
    plt.show()

def plot_correlation_matrix(stock_data: pd.DataFrame) -> None:
    """Plot the correlation matrix of the stock data features."""
    features = [col for col in stock_data.columns if col not in ['date', 'Name']]
    corr_matrix = stock_data[features].corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Correlation Matrix')
    plt.show()

def clean_data(data: pd.DataFrame) -> pd.DataFrame:
    """Fill missing values in the data."""
    data = data.fillna(method='ffill')
    logger.info("Data cleaned with forward fill.")
    return data

def calculate_rsi(prices: pd.Series, window: int = 14) -> pd.Series:
    """Calculate the Relative Strength Index (RSI) for a given series of prices."""
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(prices: pd.Series, short_period: int = 12, long_period: int = 26, signal_period: int = 9) -> Tuple[pd.Series, pd.Series]:
    """Calculate the Moving Average Convergence Divergence (MACD) for a given series of prices."""
    short_ema = prices.ewm(span=short_period, adjust=False).mean()
    long_ema = prices.ewm(span=long_period, adjust=False).mean()
    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, signal_line

def calculate_bollinger_bands(prices: pd.Series, window: int = 20, num_std: int = 2) -> Tuple[pd.Series, pd.Series]:
    """Calculate Bollinger Bands for a given series of prices."""
    rolling_mean = prices.rolling(window=window).mean()
    rolling_std = prices.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    return upper_band, lower_band

def engineer_features(data: pd.DataFrame) -> pd.DataFrame:
    """Engineer features for the stock data."""
    if data['volume'].skew() > 1:
        data['log_volume'] = np.log1p(data['volume'])
    data['prev_close'] = data['close'].shift(1)
    ma_windows = [7, 14, 21]
    for window in ma_windows:
        data[f'{window}_day_ma'] = data['close'].rolling(window=window).mean()
    data['rsi'] = calculate_rsi(data['close'])
    data['macd'], data['signal_line'] = calculate_macd(data['close'])
    data['bollinger_upper'], data['bollinger_lower'] = calculate_bollinger_bands(data['close'])
    data['pct_change'] = data['close'].pct_change()
    data['volatility'] = data['close'].rolling(window=14).std()
    
    # Lag features
    for lag in range(1, 4):
        data[f'lag_{lag}'] = data['close'].shift(lag)
        
    data.dropna(inplace=True)
    logger.info("Features engineered.")
    return data

class OutlierHandler(BaseEstimator, TransformerMixin):
    def __init__(self, threshold: float = 3.0):
        self.threshold = threshold

    def fit(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> 'OutlierHandler':
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.copy()
        for col in X.columns:
            z_scores = zscore(X[col])
            X[col] = np.where(np.abs(z_scores) > self.threshold, np.median(X[col]), X[col])
        return X

def scale_features(data: pd.DataFrame) -> Tuple[pd.DataFrame, RobustScaler]:
    """Scale features using RobustScaler."""
    scaler = RobustScaler()
    features = ['open', 'high', 'low', 'close', 'volume']
    data[features] = scaler.fit_transform(data[features])
    logger.info("Features scaled.")
    return data, scaler

def prepare_data(filepath: Union[str, Path], stock_name: str) -> Tuple[Optional[pd.DataFrame], Optional[RobustScaler]]:
    """Prepare data for modeling: load, clean, engineer features, and scale."""
    data = load_data(filepath)
    if data is None:
        return None, None
    stock_data = data[data['Name'] == stock_name].copy()
    stock_data = clean_data(stock_data)
    stock_data = engineer_features(stock_data)
    stock_data, scaler = scale_features(stock_data)
    return stock_data, scaler

def save_transformed_data(stock_data: pd.DataFrame, filename: Union[str, Path]) -> None:
    """Save the transformed stock data to a CSV file."""
    stock_data.to_csv(filename, index=False)
    logger.info(f"Transformed data saved to {filename}")

def prepare_time_series_data(data: pd.DataFrame, target: str = 'close') -> pd.DataFrame:
    """Prepare the data for time series modeling."""
    data.set_index('date', inplace=True)
    data.sort_index(inplace=True)
    return data

def evaluate_model(true: pd.Series, predicted: np.ndarray) -> Tuple[float, float]:
    """Evaluate the model performance."""
    mse = mean_squared_error(true, predicted)
    mae = mean_absolute_error(true, predicted)
    return mse, mae

def time_series_cv(model: Union[BaseEstimator, Sequential], X: pd.DataFrame, y: pd.Series, n_splits: int = 3) -> Tuple[float, float]:
    """Perform time series cross-validation."""
    tscv = TimeSeriesSplit(n_splits=n_splits)
    mse_scores, mae_scores = [], []

    if hasattr(model, 'get_params'):  # Check if the model is a scikit-learn estimator
        for train_idx, test_idx in tscv.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mse_scores.append(mean_squared_error(y_test, y_pred))
            mae_scores.append(mean_absolute_error(y_test, y_pred))
    else:  # Handle non-scikit-learn models (e.g., LSTM)
        for train_idx, test_idx in tscv.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            X_train_reshaped = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
            early_stopping = EarlyStopping(monitor='val_loss', patience=5)
            model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, verbose=0, validation_split=0.2, callbacks=[early_stopping])

            X_test_reshaped = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))
            y_pred = model.predict(X_test_reshaped)
            mse_scores.append(mean_squared_error(y_test, y_pred.flatten()))
            mae_scores.append(mean_absolute_error(y_test, y_pred.flatten()))

    mse = np.mean(mse_scores)
    mae = np.mean(mae_scores)
    logger.info(f"Cross-validation results - MSE: {mse:.4f}, MAE: {mae:.4f}")
    return mse, mae

def train_prophet(data: pd.DataFrame) -> Tuple[Prophet, float, float]:
    """Train a Prophet model."""
    prophet_data = data.reset_index().rename(columns={'date': 'ds', 'close': 'y'})
    model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True)
    model.fit(prophet_data)
    future = model.make_future_dataframe(periods=len(data), freq='D')
    forecast = model.predict(future)
    predicted = forecast['yhat'].values[-len(data):]
    mse, mae = evaluate_model(data['close'], predicted)
    logger.info(f"Prophet model trained. MSE: {mse:.4f}, MAE: {mae:.4f}")
    return model, mse, mae

def train_auto_arima(data: pd.DataFrame) -> Tuple[pm.ARIMA, float, float]:
    """Train an Auto-ARIMA model."""
    model = pm.auto_arima(data['close'], seasonal=True, m=12)
    forecast = model.predict(n_periods=len(data))
    mse, mae = evaluate_model(data['close'], forecast)
    logger.info(f"Auto-ARIMA model trained. MSE: {mse:.4f}, MAE: {mae:.4f}")
    return model, mse, mae

def train_linear_regression(X: pd.DataFrame, y: pd.Series) -> Tuple[Ridge, float, float]:
    """Train a Linear Regression model with regularization."""
    model = Ridge(alpha=1.0)  # Using Ridge regression for regularization
    model.fit(X, y)
    mse, mae = time_series_cv(model, X, y)
    logger.info(f"Linear Regression model trained with Ridge regularization. MSE: {mse:.4f}, MAE: {mae:.4f}")
    return model, mse, mae

def train_xgboost(X: pd.DataFrame, y: pd.Series) -> Tuple[XGBRegressor, float, float]:
    """Train an XGBoost model."""
    param_grid = {
        'n_estimators': [100, 300, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    }
    model = XGBRegressor()
    search = RandomizedSearchCV(model, param_grid, cv=3, n_iter=10, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
    search.fit(X, y)
    best_model = search.best_estimator_
    mse, mae = time_series_cv(best_model, X, y)
    logger.info(f"XGBoost model trained with best params: {search.best_params_}. MSE: {mse:.4f}, MAE: {mae:.4f}")
    return best_model, mse, mae

def train_lightgbm(X: pd.DataFrame, y: pd.Series) -> Tuple[LGBMRegressor, float, float]:
    """Train a LightGBM model."""
    param_grid = {
        'n_estimators': [100, 300, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'num_leaves': [31, 50, 70],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    }
    model = LGBMRegressor()
    search = RandomizedSearchCV(model, param_grid, cv=3, n_iter=10, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42, verbose=-1)  # Suppress detailed logs
    search.fit(X, y)
    best_model = search.best_estimator_
    mse, mae = time_series_cv(best_model, X, y)
    logger.info(f"LightGBM model trained with best params: {search.best_params_}. MSE: {mse:.4f}, MAE: {mae:.4f}")
    return best_model, mse, mae

def train_lstm(X: pd.DataFrame, y: pd.Series) -> Tuple[Sequential, float, float]:
    """Train an LSTM model."""
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(1, X.shape[1])))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    X_reshaped = X.values.reshape((X.shape[0], 1, X.shape[1]))

    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X_reshaped, y, epochs=100, batch_size=32, verbose=0, validation_split=0.2, callbacks=[early_stopping])
    mse, mae = time_series_cv(model, X, y)
    logger.info(f"LSTM model trained. MSE: {mse:.4f}, MAE: {mae:.4f}")
    return model, mse, mae

def train_svr(X: pd.DataFrame, y: pd.Series) -> Tuple[SVR, float, float]:
    """Train an SVR model."""
    param_grid = {
        'C': [0.1, 1.0, 10],
        'epsilon': [0.01, 0.1, 0.2],
        'kernel': ['rbf']
    }
    model = SVR()
    search = RandomizedSearchCV(model, param_grid, cv=3, n_iter=10, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
    search.fit(X, y)
    best_model = search.best_estimator_
    mse, mae = time_series_cv(best_model, X, y)
    logger.info(f"SVR model trained with best params: {search.best_params_}. MSE: {mse:.4f}, MAE: {mae:.4f}")
    return best_model, mse, mae

def select_features(X: pd.DataFrame, y: pd.Series, k: int = 10) -> pd.DataFrame:
    """Select the top k features using SelectKBest."""
    selector = SelectKBest(f_regression, k=k)
    selector.fit(X, y)
    selected_features = X.columns[selector.get_support()]
    logger.info(f"Selected features: {selected_features}")
    return X[selected_features]

def train_ensemble(models: list, X: pd.DataFrame, y: pd.Series) -> Tuple[VotingRegressor, float, float]:
    """Train an ensemble model."""
    ensemble = VotingRegressor(estimators=models)
    mse, mae = time_series_cv(ensemble, X, y)
    logger.info(f"Ensemble model trained. MSE: {mse:.4f}, MAE: {mae:.4f}")
    return ensemble, mse, mae

def compare_models(models: Dict[str, Tuple[Any, float, float]]) -> None:
    """Compare the performance of different models."""
    model_names, mse_scores, mae_scores = zip(*[(name, mse, mae) for name, (_, mse, mae) in models.items()])

    plt.figure(figsize=(10, 6))
    plt.bar(model_names, mse_scores)
    plt.xlabel('Model')
    plt.ylabel('Mean Squared Error (MSE)')
    plt.title('Model Comparison - MSE')
    plt.xticks(rotation=45)
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.bar(model_names, mae_scores)
    plt.xlabel('Model')
    plt.ylabel('Mean Absolute Error (MAE)')
    plt.title('Model Comparison - MAE')
    plt.xticks(rotation=45)
    plt.show()

def analyze_feature_importance(model: Any, feature_names: list) -> None:
    """Analyze and plot feature importance for models that support it."""
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1]
        plt.figure(figsize=(10, 6))
        plt.title('Feature Importances')
        plt.bar(range(len(importances)), importances[indices], align='center')
        plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45)
        plt.xlabel('Feature')
        plt.ylabel('Importance')
        plt.show()
    elif hasattr(model, 'coef_'):
        importances = model.coef_
        indices = np.argsort(np.abs(importances))[::-1]
        plt.figure(figsize=(10, 6))
        plt.title('Feature Importances')
        plt.bar(range(len(importances)), importances[indices], align='center')
        plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45)
        plt.xlabel('Feature')
        plt.ylabel('Importance')
        plt.show()
    else:
        logger.info("Model does not support feature importance analysis.")

def analyze_residuals(true: pd.Series, predicted: np.ndarray) -> None:
    """Analyze residuals of the model predictions."""
    residuals = true - predicted
    plt.figure(figsize=(10, 6))
    plt.hist(residuals, bins=50)
    plt.title('Residuals Distribution')
    plt.xlabel('Residual')
    plt.ylabel('Frequency')
    plt.show()
    
    plt.figure(figsize=(10, 6))
    plt.scatter(true, residuals)
    plt.title('Residuals vs True Values')
    plt.xlabel('True Values')
    plt.ylabel('Residuals')
    plt.axhline(0, color='r', linestyle='--')
    plt.show()

def main() -> None:
    """Main function to execute the modeling process."""
    filepath = 'all_stocks_5yr.csv'
    top_stocks = ['AAPL']

    # Configure logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
    logging.getLogger('prophet').setLevel(logging.WARNING)
    logging.getLogger('lightgbm').setLevel(logging.WARNING)

    for stock_name in top_stocks:
        logger.info(f"Analyzing {stock_name}:")
        stock_data, scaler = prepare_data(filepath, stock_name)
        if stock_data is None:
            logger.error("Error preparing data.")
            continue

        # Data exploration
        exploration_report = explore_data(stock_data)
        
        # Visualize the data
        visualize_data(stock_data, stock_name)
        plot_correlation_matrix(stock_data)
        
        # Save the transformed data
        save_transformed_data(stock_data, f"{stock_name}_transformed.csv")

        # Prepare time series data
        time_series_data = prepare_time_series_data(stock_data)
        X = time_series_data.drop(['close', 'Name'], axis=1)  # Drop 'close' and 'Name' columns
        y = time_series_data['close']

        # Select best features
        X_selected = select_features(X, y)

        # Train models and log their performance
        models = {
            'Prophet': train_prophet(time_series_data),
            'Auto-ARIMA': train_auto_arima(time_series_data),
            'Linear Regression': train_linear_regression(X_selected, y),
            'XGBoost': train_xgboost(X_selected, y),
            'LightGBM': train_lightgbm(X_selected, y),
            'LSTM': train_lstm(X_selected, y),
            'SVR': train_svr(X_selected, y),
        }

        logger.info("Model Performance:")
        for name, (_, mse, mae) in models.items():
            logger.info(f"{name} - MSE: {mse:.4f}, MAE: {mae:.4f}")

        # Compare model performances
        compare_models(models)

        # Train ensemble model (excluding LSTM as it is not sklearn compatible)
        ensemble_model, ensemble_mse, ensemble_mae = train_ensemble([
            ('linear', models['Linear Regression'][0]),
            ('xgboost', models['XGBoost'][0]),
            ('lightgbm', models['LightGBM'][0])
        ], X_selected, y)
        logger.info(f"Ensemble - MSE: {ensemble_mse:.4f}, MAE: {ensemble_mae:.4f}")

        # Analyze feature importance for selected models
        analyze_feature_importance(models['XGBoost'][0], X_selected.columns)
        analyze_feature_importance(models['LightGBM'][0], X_selected.columns)
        analyze_feature_importance(models['Linear Regression'][0], X_selected.columns)

        # Analyze residuals for the ensemble model
        ensemble_pred = ensemble_model.predict(X_selected)
        analyze_residuals(y, ensemble_pred)

if __name__ == "__main__":
    main()

